# Investment crowdfunding has little faith in sustainability! At least for the moment.

## Journal:

*Venture Capital*, 25:1, 91-115, 2023, [DOI: 10.1080/13691066.2022.2129510](https://doi.org/10.1080/13691066.2022.2129510)

## Authors:

Carmen Mendoza

Isabel María Parra Oller

Álvaro Rezola (@alvarorezola)

Nuria Suárez

In [23]:
# Import libraries
import pandas as pd
import locale
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from sklearn.linear_model import LogisticRegression
from statsmodels.api import add_constant
from statsmodels.discrete.discrete_model import Probit
from statsmodels.discrete.discrete_model import Probit
import math

In [24]:
# Function definition
def summary_with_stars(model):
    # get the summary table as a DataFrame
    summary_df = model.summary2().tables[1]
    
    # create a new column for stars
    summary_df['stars'] = '-'

    # add stars based on p-values
    summary_df.loc[summary_df['P>|z|'] < 0.001, 'stars'] = '***'
    summary_df.loc[(summary_df['P>|z|'] >= 0.001) & (summary_df['P>|z|'] < 0.01), 'stars'] = '**'
    summary_df.loc[(summary_df['P>|z|'] >= 0.01) & (summary_df['P>|z|'] < 0.05), 'stars'] = '*'

    # return the modified summary table
    return summary_df

def confidence_interval(X1, X2):
    #calculate degrees of freedom
    df = len(X1) + len(X2) - 2

    #calculate standard error
    s1 = np.var(X1, ddof=1)
    s2 = np.var(X2, ddof=1)
    n1 = len(X1)
    n2 = len(X2)
    standard_error = np.sqrt((s1/n1) + (s2/n2))

    #calculate margin of error
    margin_of_error = stats.t.ppf(0.95, df) * standard_error

    #calculate confidence interval
    lower_limit = (np.mean(X1) - np.mean(X2)) - margin_of_error
    upper_limit = (np.mean(X1) - np.mean(X2)) + margin_of_error

    return lower_limit, upper_limit

# calculamos el logit del puntuaje de propensión para emparejar
def logit(p):
    logit_value = math.log(p/(1 - p))
    return logit_value

# calcular standard errors ATE
def ate_se(df):
    # obtener el numero de observaciones de cada grupo
    n_t = df[df.sustainable == 1].shape[0]
    n_c = df[df.sustainable == 0].shape[0]
    
    # obtener la varianza de los resultados en cada grupo
    s_t = df[df.sustainable == 1].exito.var()
    s_c = df[df.sustainable == 0].exito.var()
    
    # calcular error estandar del ATE usando la fórmula
    se = np.sqrt((s_t / n_t) + (s_c / n_c))
    
    return se

In [ ]:
# Import data
stata_dataset = "/home/alvaro/Desktop/MendozaEtAl2023-VC/Data/CROWD_SUSTAINABILITY_FINAL.dta"
df = pd.read_stata(stata_dataset)

# Prepare data-set
df.drop(index=range(3679,len(df)), inplace=True)
df = df[df["form_c"] == 1]
df["deadline"] = pd.to_datetime(df["deadline"])
df["datestart"] = pd.to_datetime(df["datestart"])
df["dateincorporation"] = pd.to_datetime(df["dateincorporation"])
df = df[df["deadline"] <= "2019-10-01"]
df["totalassetsmostrecent1"] += 1e-6
df["logtotalassetsmostrecent1"] = np.log(df["totalassetsmostrecent1"])
df = df.dropna(subset=["desastre6meses"])

# Bank Net Income to Total Assets as value of the ratio net income-to-total assets in the banking sector per state and year.
df["bank_netincome_assets"] = df["lagnetinc"]/df["lagasset"]*100
df['bank_netincome_assets'].fillna(0, inplace=True)
df['banknetincome_assets_number'] = df['bank_netincome_assets']*df['lagbanks']
df['banknetincome_assets_number'].fillna(0, inplace=True)

# Treatment and control dataframes
df_sust = df[df["sustainable"] == 1]
df_non_sust = df[df["sustainable"] == 0]

# Fixed effect variables
year_dummies = ["yr2c"]
for i in range(3, 19):
    year_dummies.append(f"yr{i}c")
industry_dummies = ["ind2c"]
for i in range(3, 69):
    industry_dummies.append(f"ind{i}c")
state_dummies = ["state2c"]
for i in range(3, 63):
    state_dummies.append(f"state{i}c")

There are 1,766 investment crowdfunding campaings issued under the Form C exemption from May 1st, 2016 until September 10th, 2019.

### Table 1 (a): Descriptive statistics
This table shows the descriptive statistics – mean, standard deviation, 25th percentile, median, 75th percentile – of the main variables of interest.

In [26]:
df[["exito",
    "quick75relative",
    "sustainable",
    "totalassetsmostrecent1",
    "employees",
    "loglife",                     # Age as in np.log(date_diff(start, incorporation))              
    "equity",                     
    "asked",
    "lagbranches",                 # Bank branches
    "bank_netincome_assets",       # Ratio Net Income to Total Assets (after we will use banknetincome_assets_number)
    "lagvcfundraising",            # VC fundraising (not sure if then I need to do the np.log())
    "loglagnum_oper_por_platf_y",  # Number of offerings per platform
    ]].describe()

,exito,quick75relative,sustainable,totalassetsmostrecent1,employees,loglife,equity,asked,lagbranches,bank_netincome_assets,lagvcfundraising,loglagnum_oper_por_platf_y
count,1766.000000,1766.000000,1766.000000,1.766000e+03,1766.000000,1765.000000,1766.000000,1.766000e+03,1760.000000,1766.000000,1581.000000,1765.000000
mean,0.346546,0.026614,0.151755,3.346530e+05,5.361835,6.102006,0.270102,7.045705e+04,3968.019318,1.037196,8813.899414,67.960342
std,0.476001,0.160998,0.358885,1.762435e+06,10.314373,1.464374,0.444140,1.185048e+05,2359.036538,0.558752,11128.952148,65.647285
min,0.000000,0.000000,0.000000,1.000000e-06,0.000000,0.690000,0.000000,1.000000e+03,117.000000,0.000000,0.620000,2.000000
25%,0.000000,0.000000,0.000000,1.000000e-06,1.000000,5.320000,0.000000,1.000000e+04,1484.000000,0.818707,90.800003,9.000000
50%,0.000000,0.000000,0.000000,2.656100e+04,3.000000,6.350000,0.000000,2.500000e+04,4260.000000,0.885748,943.030029,37.000000
75%,1.000000,0.000000,0.000000,2.032070e+05,6.000000,7.150000,1.000000,7.875000e+04,6728.000000,1.091834,20763.099609,110.000000
max,1.000000,1.000000,1.000000,5.812937e+07,225.000000,9.870000,1.000000,1.070000e+06,6868.000000,4.925659,33174.179688,262.000000


 ### Table 1 (b): Mean differences across subsamples
 This table shows the mean values of the main variables across the two subsamples of offerings and the T-statistic for the mean differences. The T-statistics reported are obtained for the differences between the means across groups of offerings. All the variables are defined in Annex 1. ***, **, and * indicate statistical significance at 1, 5, and 10 percent, respectively.

In [27]:
data = {
    "success": (df_non_sust["exito"],df_sust["exito"]),
    "quick75relative": (df_non_sust["quick75relative"],df_sust["quick75relative"]),
    "totalassetsmostrecent1": (df_non_sust["totalassetsmostrecent1"],df_sust["totalassetsmostrecent1"]),
    "employees": (df_non_sust["employees"],df_sust["employees"]),
    "loglife": (df_non_sust["loglife"],df_sust["loglife"]),
    "equity": (df_non_sust["equity"],df_sust["equity"]),
    "asked": (df_non_sust["asked"],df_sust["asked"]),
    "lagbranches": (df_non_sust["lagbranches"],df_sust["lagbranches"]),
    "banknetincome_assets_number": (df_non_sust["banknetincome_assets_number"], df_sust["banknetincome_assets_number"]),
    "lagvcfundraising": (df_non_sust["lagvcfundraising"],df_sust["lagvcfundraising"]),
    "loglagnum_oper_por_platf_y": (df_non_sust["loglagnum_oper_por_platf_y"],df_sust["loglagnum_oper_por_platf_y"]),
}

results = {}

for key in data:
    group1 = data[key][0]
    group2 = data[key][1]

    # calculate t-statistic    
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy="omit")
    
    # Indicate statistical significance at different levels
    if p_value < 0.01:
        significance = "***"
    elif p_value < 0.05:
        significance = "**"
    elif p_value < 0.10:
        significance = "*"
    else:
        significance = "-"
        
    # calculate mean of each variable
    mean_group1 = np.mean(group1)
    mean_group2 = np.mean(group2)

    results[key]={'non sustainable':mean_group1,
                  'sustainable':mean_group2,
                  't-statistic':t_statistic,
                  'p-value':p_value,
                  'significance':significance}
# Display results
pd.DataFrame(results).T

# print(df_results)

,non sustainable,sustainable,t-statistic,p-value,significance
success,0.341121,0.376866,-1.132289,0.257667,-
quick75relative,0.02737,0.022388,0.46644,0.640958,-
totalassetsmostrecent1,327075.5,377007.5625,-0.427066,0.669383,-
employees,5.491989,4.634328,1.253924,0.210036,-
loglife,6.067301,6.295858,-2.356181,0.018573,**
equity,0.261682,0.317164,-1.884844,0.059615,*
asked,70123.804688,72319.789062,-0.279324,0.780029,-
lagbranches,4045.154722,3536.696629,3.252588,0.001165,***
banknetincome_assets_number,139.827187,107.90303,3.804976,0.000147,***
lagvcfundraising,8817.848633,8790.697266,0.034193,0.972728,-


### Table 2: Propensity Score Matching 
This table shows the mean values of the main variables across the two subsamples of offerings and the t-statistics obtained for the differences between the means across groups of offerings, before matching and after matching using caliper, nearest 1-to-1 and nn-VBC methods. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

In [28]:
# Test for heteroskedascity to understand if we should use robust standard errors

In [29]:
# Two probit functions to validate covariates
formula_traetment = f"sustainable ~ logtotalassetsmostrecent1 + logemployees1 + logasked1"
summary_with_stars(smf.probit(formula=formula_traetment, data=df).fit(cov_type='HC0'))

Optimization terminated successfully.
         Current function value: 0.424215
         Iterations 5


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],stars
Intercept,-0.398424,0.355543,-1.120606,0.262456,-1.095275,0.298428,-
logtotalassetsmostrecent1,0.004643,0.003503,1.325460,0.185019,-0.002223,0.011509,-
logemployees1,-0.075966,0.045489,-1.669971,0.094925,-0.165123,0.013192,-
logasked1,-0.052412,0.033573,-1.561123,0.118495,-0.118213,0.013390,-


In [30]:
formula_outcome = f"exito ~ logtotalassetsmostrecent1 + logemployees1 + logasked1"
summary_with_stars(smf.probit(formula=formula_outcome, data=df).fit(cov_type='HC0'))

Optimization terminated successfully.
         Current function value: 0.626012
         Iterations 5


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],stars
Intercept,1.497462,0.289416,5.174074,2.290439e-07,0.930216,2.064708,***
logtotalassetsmostrecent1,0.003466,0.003018,1.148521,2.507537e-01,-0.002449,0.009382,-
logemployees1,0.127192,0.039222,3.242845,1.183428e-03,0.050318,0.204066,**
logasked1,-0.201891,0.027189,-7.425398,1.124414e-13,-0.255181,-0.148601,***


### Table 2 (A): Before Matching

In [31]:
df[["exito", "quick75relative", "logtotalassetsmostrecent1", "logemployees1", "logasked1"]].describe()
df_sust = df[df["sustainable"] == 1]
df_non_sust = df[df["sustainable"] == 0]

data = {
    "success": (df_sust["exito"], df_non_sust["exito"]),
    "quick75relative": (df_sust["quick75relative"], df_non_sust["quick75relative"]),
    "logtotalassetsmostrecent1": (df_sust["logtotalassetsmostrecent1"], df_non_sust["logtotalassetsmostrecent1"]),
    "logemployees1": (df_sust["logemployees1"], df_non_sust["logemployees1"]),
    "logasked1": (df_sust["logasked1"], df_non_sust["logasked1"])
}

results = {}

for key in data:
    group1 = data[key][0]
    group2 = data[key][1]

    # calculate t-statistic    
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy="omit")
    
    # Indicate statistical significance at different levels
    if p_value < 0.01:
        significance = "***"
    elif p_value < 0.05:
        significance = "**"
    elif p_value < 0.10:
        significance = "*"
    else:
        significance = "-"
        
    # calculate mean of each variable
    mean_group1 = np.mean(group1)
    mean_group2 = np.mean(group2)

    results[key]={'sustainable':mean_group1,
                  'non_sustainable':mean_group2,
                  "difference" :(mean_group1 - mean_group2),
                  't-statistic':t_statistic,
                  'p-value':p_value,
                  'significance':significance}
# Display results
pd.DataFrame(results).T

,sustainable,non_sustainable,difference,t-statistic,p-value,significance
success,0.376866,0.341121,0.035744,1.132289,0.257667,-
quick75relative,0.022388,0.02737,-0.004982,-0.46644,0.640958,-
logtotalassetsmostrecent1,4.733572,4.211653,0.52192,0.699538,0.484308,-
logemployees1,1.36893,1.434692,-0.065762,-1.174444,0.240376,-
logasked1,10.326214,10.433923,-0.107709,-1.415024,0.157238,-


### Table 2 (B): After Matching

In [32]:
# Logit model para estimar el puntaje de propensión (ps)
model = LogisticRegression()
df = df.dropna(axis = 1) # drop all variables that have empty values

# Independent variables
X = df[["logtotalassetsmostrecent1",
        "logemployees1",
        "logasked1"]]

# Dependent variable (treatment group)
y = df["sustainable"]

# Model adjustment & predicted probabilities
model.fit(X, y) 
pred_prob = model.predict_proba(X)
df["ps"] = pred_prob[:, 1]

df["ps_logit"] = df.ps.apply(logit)

# Implementing the caliper match
def caliper_match(df, threshold):
    # ordenar los datos por ps_logit y crear una columna con el índice original
    df_sorted = df.sort_values("ps_logit").reset_index()
    df_sorted["orig_index"] = df_sorted.index
    
    # crear listas vacias para almacenar los indices emparejados y no emparejados
    matched_index = []
    unmatched_index = []
    
    # iterar sobre las filas del dataframe ordenado
    for i in range(len(df_sorted)):
        row = df_sorted.iloc[i]
        if i not in matched_index: # si la fila no está emparejada todavía
            potential_matches = df_sorted[(df_sorted.sustainable != row.sustainable) & (abs(df_sorted.ps_logit - row.ps_logit) <= threshold)]
            
            # encontrar las filas potenciales que tienen un tratamiento diferente y una diferencia de ps_logit menor o igual al umbral
            
            if len(potential_matches) > 0: # si hay al menos una fila potencialmente emparejable
                closest_match_index = potential_matches.iloc[0].orig_index
                # Tomar la priemra fila potencial como la más cercana
                
                matched_index.append(i)
                matched_index.append(closest_match_index)
                # añadir ambos índices a la lista de emparejados
                
            else:
                unmatched_index.append(i)
                # si no hay ninguna fila potencialmente emparejable, añadir el índice a la lista d eno emparejados
    return matched_index, unmatched_index

caliper_matched, caliper_unmatched = caliper_match(df[["sustainable", "logtotalassetsmostrecent1", "logemployees1", "logasked1", "ps_logit"]], 0.1) # umbral 0.2
caliper_df_matched = df.iloc[caliper_matched]

# mean of each variable in the treatment & control group
treatment_means_caliper = caliper_df_matched[caliper_df_matched["sustainable"] == 1]
control_means_caliper = caliper_df_matched[caliper_df_matched["sustainable"] == 0]

data = {
    "success": (treatment_means_caliper["exito"], control_means_caliper["exito"]),
    "quick75relative": (treatment_means_caliper["quick75relative"], control_means_caliper["quick75relative"]),
    "logtotalassetsmostrecent1": (treatment_means_caliper["logtotalassetsmostrecent1"], control_means_caliper["logtotalassetsmostrecent1"]),
    "logemployees1": (treatment_means_caliper["logemployees1"], control_means_caliper["logemployees1"]),
    "logasked1": (treatment_means_caliper["logasked1"], control_means_caliper["logasked1"])
}

results = {}

for key in data:
    group1 = data[key][0]
    group2 = data[key][1]

    # calculate t-statistic    
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy="omit")
    
    # Indicate statistical significance at different levels
    if p_value < 0.01:
        significance = "***"
    elif p_value < 0.05:
        significance = "**"
    elif p_value < 0.10:
        significance = "*"
    else:
        significance = "-"
        
    # calculate mean of each variable
    mean_group1 = np.mean(group1)
    mean_group2 = np.mean(group2)

    results[key]={'sustainable':mean_group1,
                  'non_sustainable':mean_group2,
                  "difference" :(mean_group1 - mean_group2),
                  't-statistic':t_statistic,
                  'p-value':p_value,
                  'significance':significance}
# Display results
pd.DataFrame(results).T

,sustainable,non_sustainable,difference,t-statistic,p-value,significance
success,0.340564,0.306615,0.033949,1.468236,0.142129,-
quick75relative,0.017354,0.020528,-0.003174,-0.45255,0.650901,-
logtotalassetsmostrecent1,5.069388,5.53946,-0.470072,-0.902258,0.366981,-
logemployees1,1.318171,1.482502,-0.16433,-4.057515,0.000051,***
logasked1,10.476681,10.427021,0.04966,0.890431,0.373295,-


### Table 3: Average Treatment Effect on the Treated (ATET) 
This table shows the average treatment effect on the treated individuals (ATET) for each method: caliper, nearest 1-to-1 and nn-VBC methods. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

In [33]:
caliper_ate_success = caliper_df_matched.groupby("sustainable")["exito"].mean().diff().iloc[-1]
caliper_ate_quick = caliper_df_matched.groupby("sustainable")["quick75relative"].mean().diff().iloc[-1]
caliper_se = ate_se(caliper_df_matched)

# realizar una preuba de t de dos muestras independientes y obtener los valores p y los intervalso de confianza al 95%
caliper_success_tstat, caliper_success_pvalue, caliper_success_desc = sms.ttest_ind(
    caliper_df_matched[caliper_df_matched.sustainable == 1].exito.values,
    caliper_df_matched[caliper_df_matched.sustainable == 0].exito.values,
    usevar="unequal",
    alternative="larger",
    value=0
)
lower_limit_success, upper_limit_success = confidence_interval(caliper_df_matched[caliper_df_matched.sustainable == 1].exito.values,
                                                               caliper_df_matched[caliper_df_matched.sustainable == 0].exito.values)

# realizar una preuba de t de dos muestras independientes y obtener los valores p y los intervalso de confianza al 95%
caliper_quick_tstat, caliper_quick_pvalue, caliper_quick_desc = sms.ttest_ind(
    caliper_df_matched[caliper_df_matched.sustainable == 1].quick75relative.values,
    caliper_df_matched[caliper_df_matched.sustainable == 0].quick75relative.values,
    usevar="unequal",
    alternative="larger",
    value=0
)

lower_limit_quick, upper_limit_quick = confidence_interval(caliper_df_matched[caliper_df_matched.sustainable == 1].quick75relative.values,
                                                           caliper_df_matched[caliper_df_matched.sustainable == 0].quick75relative.values)

data = {"Modelo caliper 1-to-1": ["success", "quickrelative75"],
        "ATE": [caliper_ate_success, caliper_ate_quick],
        "Error estándar": [caliper_se, caliper_se],
        "Valor p": [caliper_success_pvalue, caliper_quick_pvalue],
        "[Intervalo de": [lower_limit_success, lower_limit_quick],
        "confianza (95%)]": [upper_limit_success, upper_limit_quick]}

pd.DataFrame(data)

,Modelo caliper 1-to-1,ATE,Error estándar,Valor p,[Intervalo de,confianza (95%)]
0,success,0.033949,0.023612,0.075504,-0.004899,0.072798
1,quickrelative75,-0.003174,0.023612,0.684516,-0.014041,0.007693


### Table 4: Sustainability and success
This table presents IV results examining the effect of the sustainable orientation of investment crowdfunding offerings on the probability of success. The dependent variable in columns (1) and (2) is the dummy that identifies sustainable offerings (Sustainable). The dependent variable in columns (3) and (5) is SUCCESS. QUICK75 is the dependent variables in columns (4) and (6). Variables definitions are reported in Annex 1. Year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

In [34]:
columns = ['desastre6meses',
           'sizemostrecent1',
           'logemployees1',
           'edad',
           'equity',
           'logasked1',
           'banknetincome_assets_number',
           "ind_year",
           "state_id"] + year_dummies
X = df[columns]
y = df['sustainable']

# Drop dummies that have a constant value for all rows
nunique = X[year_dummies].nunique()
X = X.drop(columns = nunique[nunique == 1].index)

# fit model
model = Probit(y, X)
summary_with_stars(model.fit(cov_type='HC0'))

Optimization terminated successfully.
         Current function value: 0.403420
         Iterations 6


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],stars
desastre6meses,-0.223087,0.097135,-2.296676,2.163725e-02,-0.413467,-0.032706,*
sizemostrecent1,-0.003879,0.009151,-0.423865,6.716640e-01,-0.021813,0.014056,-
logemployees1,-0.116127,0.048065,-2.416034,1.569061e-02,-0.210333,-0.021921,*
edad,0.079726,0.031889,2.500130,1.241478e-02,0.017225,0.142226,*
equity,0.179842,0.087312,2.059775,3.942003e-02,0.008715,0.350970,*
logasked1,-0.032543,0.036804,-0.884221,3.765770e-01,-0.104677,0.039591,-
banknetincome_assets_number,-0.000916,0.000365,-2.506893,1.217974e-02,-0.001631,-0.000200,*
ind_year,-0.000770,0.000137,-5.607963,2.047215e-08,-0.001040,-0.000501,***
state_id,0.000514,0.002103,0.244620,8.067505e-01,-0.003608,0.004636,-
yr15c,-0.325705,0.503342,-0.647085,5.175772e-01,-1.312237,0.660827,-


### Table 5: Sustainability and success: the role of firm- and offering-level characteristics
This table presents results examining the effect of firm- and offering-level characteristics on the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. *** and ** indicate statistical significance at 1 and 5 percent, respectively.

In [35]:
result = model.fit(cov_type='HC0')

# Uses the fitted model to make predictions for the dependent variable (sustainable)
df['sustainablep'] = result.predict(X)

# This new columns represent an interaction term between the predicted values of sustainable and the values of XX.
df['sustainablep_lagbranches'] = df['sustainablep']*df["lagbranches"]
df['sustainablep_netincome_assetsB'] = df['sustainablep']*df["bank_netincome_assets"]
df['sustainablep_lagvcfundraising'] = df['sustainablep']*df["lagvcfundraising"]
df['sustainablep_loglagnum_op_per_platform_y'] = df['sustainablep']*df["loglagnum_oper_por_platf_y"]

# Without desastre6meses
controls = ['sustainablep',
           'sizemostrecent1',
           'logemployees1',
           'edad',
           'equity',
           "logasked1",
           "ind_year",
           "state_id"] + year_dummies

Table_51_controls = controls + "sustainablep_size"
Table_52_controls = controls + "sustainablep_employees"
Table_53_controls = controls + "sustainablep_age"
Table_54_controls = controls + "sustainablep_equity"
Table_55_controls = controls + "sustainablep_asked"

X2 = df[Table_51_controls]

# Drop dummies that have a constant value for all rows
nunique = X2[year_dummies].nunique()
X2 = X2.drop(columns = nunique[nunique == 1].index)
y2 = df['exito']

model2 = Probit(y2, X2)
summary_with_stars(model2.fit(cov_type='HC0'))

Optimization terminated successfully.
         Current function value: 0.403420
         Iterations 6


KeyError: 'lagbranches'

### Table 6: Sustainability and success: the role of the financing environment
This table presents results examining the effect of the characteristics of the financing environment on the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Firm and offering control variables, year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***; ** and * indicate statistical significance at 1, 5, and 10 percent, respectively.

In [ ]:
result = model.fit(cov_type='HC0')

# Uses the fitted model to make predictions for the dependent variable (sustainable)
df['sustainablep'] = result.predict(X)

# This new columns represent an interaction term between the predicted values of sustainable and the values of XX.
df['sustainablep_lagbranches'] = df['sustainablep']*df["lagbranches"]
df['sustainablep_netincome_assetsB'] = df['sustainablep']*df["bank_netincome_assets"]
df['sustainablep_lagvcfundraising'] = df['sustainablep']*df["lagvcfundraising"]
df['sustainablep_loglagnum_op_per_platform_y'] = df['sustainablep']*df["loglagnum_oper_por_platf_y"]

# Without desastre6meses
controls = ['sustainablep',
           'sizemostrecent1',
           'logemployees1',
           'edad',
           'equity',
           "logasked1",
           "ind_year",
           "state_id"] + year_dummies

Table_61_controls = controls + "sustainablep_lagbranches"
Table_62_controls = controls + "sustainablep_netincome_assetsB"
Table_63_controls = controls + "sustainablep_lagvcfundraising"
Table_64_controls = controls + "sustainablep_loglagnum_op_per_platform_y"

X2 = df[Table_61_controls]

# Drop dummies that have a constant value for all rows
nunique = X2[year_dummies].nunique()
X2 = X2.drop(columns = nunique[nunique == 1].index)
y2 = df['exito']

model2 = Probit(y2, X2)
summary_with_stars(model2.fit(cov_type='HC0'))

### Table 7: Sustainability and success: robustness tests
This table presents a set of robustness tests for the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. In column (1), we report the results for the second-stage regression for the Heckman (1979) method. In column (2), we find that the results do not vary when controlling for the characteristics of the team in terms of gender and size. In columns (3) to (5), we control for the funding history of the company. In column (6), we control for the cost structure defined by the funding portal. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Firm and offering control variables, year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***; ** and * indicate statistical significance at 1, 5, and 10 percent, respectively.